In [221]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

#Both the next libraries used for the prediction of the cabin
from sklearn.linear_model import LogisticRegression
#to plot
import seaborn as sns
import matplotlib.pyplot as plt
#For the accuracy 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,roc_curve
#To split 
from sklearn.model_selection import train_test_split


In [222]:
#Read Training data provided 
train_data = pd.read_csv("titanic/train.csv")
test_data = pd.read_csv("titanic/test.csv")

Data Preprocessing

In [223]:
#Display the first 5 rows of the data 
train_data.head()
test_data.head()

PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S

In [224]:
#To get the matrix of the data
#train_data.shape()
train_data.info()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [225]:
#Description of the data 
train_data.describe()
test_data.describe()

PassengerId      Pclass         Age       SibSp       Parch        Fare
count   418.000000  418.000000  332.000000  418.000000  418.000000  417.000000
mean   1100.500000    2.265550   30.272590    0.447368    0.392344   35.627188
std     120.810458    0.841838   14.181209    0.896760    0.981429   55.907576
min     892.000000    1.000000    0.170000    0.000000    0.000000    0.000000
25%     996.250000    1.000000   21.000000    0.000000    0.000000    7.895800
50%    1100.500000    3.000000   27.000000    0.000000    0.000000   14.454200
75%    1204.750000    3.000000   39.000000    1.000000    0.000000   31.500000
max    1309.000000    3.000000   76.000000    8.000000    9.000000  512.329200

In [226]:
#Check if there is missing data 
#Output is the sum of all the missing data in every coloumn 
train_data.isnull().sum()
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Age, Cabin and Embarked have missing data

In [227]:
known_ages = train_data[train_data['Age'].notnull()]
unknown_ages = train_data[train_data['Age'].isnull()]

#use Linear interpolation to fill the missing ages 
#Linear interpolation estimate missing age based on previous ages
train_data['Age'].interpolate(method='linear', inplace=True)
##########################################################################
#did the same to the test data 
known_ages_test = test_data[test_data['Age'].notnull()]
unknown_ages_test = test_data[test_data['Age'].isnull()]

#use Linear interpolation to fill the missing ages 
#Linear interpolation estimate missing age based on previous ages
test_data['Age'].interpolate(method='linear', inplace=True)

In [228]:
#check if the age data is still null/missing
train_data.isnull().sum()
test_data.isnull().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Filled all the ages, need to  fill Cabin 

In [229]:
#Sex/Geneder is categorical data so represent it in 0,1
train_data['Sex']=train_data["Sex"].map({'female':1,'male':0})
######
#Sex/Geneder is categorical data so represent it in 0,1
test_data['Sex']=test_data["Sex"].map({'female':1,'male':0})

Fill the missing data from fare 

In [230]:
#There is one null value in fare will add the mean 
fare_mean = test_data['Fare'].mean()

# Fill missing values in the "Fare" column with the mean
test_data['Fare'] = test_data['Fare'].fillna(fare_mean)

Worked on categorical data #1

In [231]:
#To help predict the cabin we need to modify another categorical column 
#The Ticket as every A/5-> 1 , PC->2, STON/O2->3, Other ->4
#Can modify by writing function to handle this case
#tickets = train_data['Ticket'].str.upper()

def label_ticket(tickets):
    if 'A/5'in tickets:
        return 1
    elif 'PC' in tickets:
        return 2
    elif 'STON/O2.' in tickets:
        return 3
    else: 
        return 4
    

#To map 
train_data['Ticket']= train_data['Ticket'].str.upper()
train_data['Ticket']= train_data['Ticket'].apply(label_ticket)
train_data.head(20)

PassengerId  Survived  Pclass  \
0             1         0       3   
1             2         1       1   
2             3         1       3   
3             4         1       1   
4             5         0       3   
5             6         0       3   
6             7         0       1   
7             8         0       3   
8             9         1       3   
9            10         1       2   
10           11         1       3   
11           12         1       1   
12           13         0       3   
13           14         0       3   
14           15         0       3   
15           16         1       2   
16           17         0       3   
17           18         1       2   
18           19         0       3   
19           20         1       3   

                                                 Name  Sex   Age  SibSp  \
0                             Braund, Mr. Owen Harris    0  22.0      1   
1   Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1   
2                              Heikkinen, Miss. Laina    1  26.0      0   
3        Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1   
4                            Allen, Mr. William Henry    0  35.0      0   
5                                    Moran, Mr. James    0  44.5      0   
6                             McCarthy, Mr. Timothy J    0  54.0      0   
7                      Palsson, Master. Gosta Leonard    0   2.0      3   
8   Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)    1  27.0      0   
9                 Nasser, Mrs. Nicholas (Adele Achem)    1  14.0      1   
10                    Sandstrom, Miss. Marguerite Rut    1   4.0      1   
11                           Bonnell, Miss. Elizabeth    1  58.0      0   
12                     Saundercock, Mr. William Henry    0  20.0      0   
13                        Andersson, Mr. Anders Johan    0  39.0      1   
14               Vestrom, Miss. Hulda Amanda Adolfina    1  14.0      0   
15                   Hewlett, Mrs. (Mary D Kingcome)     1  55.0      0   
16                               Rice, Master. Eugene    0   2.0      4   
17                       Williams, Mr. Charles Eugene    0  16.5      0   
18  Vander Planke, Mrs. Julius (Emelia Maria Vande...    1  31.0      1   
19                            Masselmani, Mrs. Fatima    1  33.0      0   

    Parch  Ticket     Fare Cabin Embarked  
0       0       1   7.2500   NaN        S  
1       0       2  71.2833   C85        C  
2       0       3   7.9250   NaN        S  
3       0       4  53.1000  C123        S  
4       0       4   8.0500   NaN        S  
5       0       4   8.4583   NaN        Q  
6       0       4  51.8625   E46        S  
7       1       4  21.0750   NaN        S  
8       2       4  11.1333   NaN        S  
9       0       4  30.0708   NaN        C  
10      1       4  16.7000    G6        S  
11      0       4  26.5500  C103        S  
12      0       1   8.0500   NaN        S  
13      5       4  31.2750   NaN        S  
14      0       4   7.8542   NaN        S  
15      0       4  16.0000   NaN        S  
16      1       4  29.1250   NaN        Q  
17      0       4  13.0000   NaN        S  
18      0       4  18.0000   NaN        S  
19      0       4   7.2250   NaN        C

In [232]:
#Did the same to test data

def label_ticket_test(tickets):
    if 'A/5'in tickets:
        return 1
    elif 'PC' in tickets:
        return 2
    elif 'STON/O2.' in tickets:
        return 3
    else: 
        return 4
    

#To map 
test_data['Ticket']= test_data['Ticket'].str.upper()
test_data['Ticket']= test_data['Ticket'].apply(label_ticket_test)
test_data.head(20)

PassengerId  Pclass                                               Name  \
0           892       3                                   Kelly, Mr. James   
1           893       3                   Wilkes, Mrs. James (Ellen Needs)   
2           894       2                          Myles, Mr. Thomas Francis   
3           895       3                                   Wirz, Mr. Albert   
4           896       3       Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
5           897       3                         Svensson, Mr. Johan Cervin   
6           898       3                               Connolly, Miss. Kate   
7           899       2                       Caldwell, Mr. Albert Francis   
8           900       3          Abrahim, Mrs. Joseph (Sophie Halaut Easu)   
9           901       3                            Davies, Mr. John Samuel   
10          902       3                                   Ilieff, Mr. Ylio   
11          903       1                         Jones, Mr. Charles Cresson   
12          904       1      Snyder, Mrs. John Pillsbury (Nelle Stevenson)   
13          905       2                               Howard, Mr. Benjamin   
14          906       1  Chaffee, Mrs. Herbert Fuller (Carrie Constance...   
15          907       2      del Carlo, Mrs. Sebastiano (Argenia Genovesi)   
16          908       2                                  Keane, Mr. Daniel   
17          909       3                                  Assaf, Mr. Gerios   
18          910       3                       Ilmakangas, Miss. Ida Livija   
19          911       3              Assaf Khalil, Mrs. Mariana (Miriam")"   

    Sex   Age  SibSp  Parch  Ticket     Fare Cabin Embarked  
0     0  34.5      0      0       4   7.8292   NaN        Q  
1     1  47.0      1      0       4   7.0000   NaN        S  
2     0  62.0      0      0       4   9.6875   NaN        Q  
3     0  27.0      0      0       4   8.6625   NaN        S  
4     1  22.0      1      1       4  12.2875   NaN        S  
5     0  14.0      0      0       4   9.2250   NaN        S  
6     1  30.0      0      0       4   7.6292   NaN        Q  
7     0  26.0      1      1       4  29.0000   NaN        S  
8     1  18.0      0      0       4   7.2292   NaN        C  
9     0  21.0      2      0       4  24.1500   NaN        S  
10    0  33.5      0      0       4   7.8958   NaN        S  
11    0  46.0      0      0       4  26.0000   NaN        S  
12    1  23.0      1      0       4  82.2667   B45        S  
13    0  63.0      1      0       4  26.0000   NaN        S  
14    1  47.0      1      0       4  61.1750   E31        S  
15    1  24.0      1      0       4  27.7208   NaN        C  
16    0  35.0      0      0       4  12.3500   NaN        Q  
17    0  21.0      0      0       4   7.2250   NaN        C  
18    1  27.0      1      0       3   7.9250   NaN        S  
19    1  45.0      0      0       4   7.2250   NaN        C

Now work with predicting the null/missing values from the Cabin column

Will use linear regression to predict the missing cabins based on the Pclass, Ticket and Fare

In [233]:
known_cabins = train_data[train_data['Cabin'].notnull()]
unknown_cabins = train_data[train_data['Cabin'].isnull()]
#	Exclude the feature that is not needed for the logistic regression 	
features= ['Fare', 'Pclass']
LR_Model = LogisticRegression()
LR_Model.fit(known_cabins[features], known_cabins['Cabin'])
predicted_cabin = LR_Model.predict(unknown_cabins[features])
train_data.loc[train_data['Cabin'].isnull(),'Cabin'] = predicted_cabin

/Users/malakelbakry/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [234]:
#train_data = pd.get_dummies(train_data,columns=['Cabin'],drop_first=True)
train_data['Cabin']= train_data['Cabin'].astype('category')
train_data['Embarked']=train_data['Embarked'].astype('category')

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    object  
 4   Sex          891 non-null    int64   
 5   Age          891 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    int64   
 9   Fare         891 non-null    float64 
 10  Cabin        891 non-null    category
 11  Embarked     889 non-null    category
dtypes: category(2), float64(2), int64(7), object(1)
memory usage: 77.7+ KB


Check if there is null/missing values in the cabin

In [235]:
train_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64

In [236]:
missing_data_rows = train_data[train_data.isnull().any(axis=1)]
missing_data_rows.head()

PassengerId  Survived  Pclass                                       Name  \
61            62         1       1                        Icard, Miss. Amelie   
829          830         1       1  Stone, Mrs. George Nelson (Martha Evelyn)   

     Sex   Age  SibSp  Parch  Ticket  Fare Cabin Embarked  
61     1  38.0      0      0       4  80.0   B28      NaN  
829    1  62.0      0      0       4  80.0   B28      NaN

Used logistic regression to predict the missing values from for embarked 

In [237]:
#train_data.head(50)
known_Embarked = train_data[train_data['Embarked'].notnull()]
unknown_Embarked = train_data[train_data['Embarked'].isnull()]
#	Exclude the feature that is not needed for the logistic regression 	
features= ['Fare', 'Pclass']
LR_Model_1 = LogisticRegression()
LR_Model_1.fit(known_Embarked[features], known_Embarked['Embarked'])
predicted_Embarked = LR_Model_1.predict(unknown_Embarked[features])
train_data.loc[train_data['Embarked'].isnull(),'Embarked'] = predicted_Embarked

train_data.head(62)

PassengerId  Survived  Pclass  \
0             1         0       3   
1             2         1       1   
2             3         1       3   
3             4         1       1   
4             5         0       3   
..          ...       ...     ...   
57           58         0       3   
58           59         1       2   
59           60         0       3   
60           61         0       3   
61           62         1       1   

                                                 Name  Sex   Age  SibSp  \
0                             Braund, Mr. Owen Harris    0  22.0      1   
1   Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1   
2                              Heikkinen, Miss. Laina    1  26.0      0   
3        Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1   
4                            Allen, Mr. William Henry    0  35.0      0   
..                                                ...  ...   ...    ...   
57                                Novel, Mr. Mansouer    0  28.5      0   
58                       West, Miss. Constance Mirium    1   5.0      1   
59                 Goodwin, Master. William Frederick    0  11.0      5   
60                              Sirayanian, Mr. Orsen    0  22.0      0   
61                                Icard, Miss. Amelie    1  38.0      0   

    Parch  Ticket     Fare Cabin Embarked  
0       0       1   7.2500    G6        S  
1       0       2  71.2833   C85        C  
2       0       3   7.9250    G6        S  
3       0       4  53.1000  C123        S  
4       0       4   8.0500    G6        S  
..    ...     ...      ...   ...      ...  
57      0       4   7.2292    G6        C  
58      2       4  27.7500    F2        S  
59      2       4  46.9000    F2        S  
60      0       4   7.2292    G6        C  
61      0       4  80.0000   B28        S  

[62 rows x 12 columns]

Check if there is any null values

In [238]:
train_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

Feature Engineering 

In [239]:
#Can generate age group to help predict the survival 
bins = [0, 10, 18, 40, 60, 100]
labels = ['Child', 'Adolescents', 'Adult', 'Middle-aged', 'Elderly']
train_data['AgeGroup'] = pd.cut(train_data['Age'], bins=bins, labels=labels, right=False)

#Can generate cabin letter BY REGULAR EXPRESSION
train_data['CabinLetter'] = train_data['Cabin'].str.extract(r'([A-Za-z])')

train_data['CabinLetter']=train_data['CabinLetter'].astype('category')


In [240]:
#Do the same for test data 
#Can generate age group to help predict the survival 
bins = [0, 10, 18, 40, 60, 100]
labels = ['Child', 'Adolescents', 'Adult', 'Middle-aged', 'Elderly']
test_data['AgeGroup'] = pd.cut(test_data['Age'], bins=bins, labels=labels, right=False)


In [241]:
#Work with categorical data 
def label_Age_Group(age):
    #Child
    if age>=0 and  age<=10:
        return 1
    #Adolescents
    elif  age>10 and  age<=18:
        return 2
    #Adult
    elif  age>18 and  age<=40:
        return 3
    #Middle aged
    elif  age>40 and  age<=60:
        return 4
    #Elderly
    elif  age>60 and  age<=100:
        return 5 
    #older than 100 years 
    else: 
        return 6
    

#To map 
train_data['AgeGroup']= train_data['AgeGroup'].str.upper()
train_data['AgeGroup']= train_data['Age'].apply(label_Age_Group)
#train_data.head(20)

In [242]:
#Do the same for test data 
#Work with categorical data 
def label_Age_Group_test(age):
    #Child
    if age>=0 and  age<=10:
        return 1
    #Adolescents
    elif  age>10 and  age<=18:
        return 2
    #Adult
    elif  age>18 and  age<=40:
        return 3
    #Middle aged
    elif  age>40 and  age<=60:
        return 4
    #Elderly
    elif  age>60 and  age<=100:
        return 5 
    #older than 100 years 
    else: 
        return 6
    

#To map 
test_data['AgeGroup']= test_data['AgeGroup'].str.upper()
test_data['AgeGroup']= test_data['Age'].apply(label_Age_Group)

In [243]:
#train_data.info()
train_data.head()
#train_data.isnull().sum()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name  Sex   Age  SibSp  Parch  \
0                            Braund, Mr. Owen Harris    0  22.0      1      0   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1      0   
2                             Heikkinen, Miss. Laina    1  26.0      0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1      0   
4                           Allen, Mr. William Henry    0  35.0      0      0   

   Ticket     Fare Cabin Embarked  AgeGroup CabinLetter  
0       1   7.2500    G6        S         3           G  
1       2  71.2833   C85        C         3           C  
2       3   7.9250    G6        S         3           G  
3       4  53.1000  C123        S         3           C  
4       4   8.0500    G6        S         3           G

Feature Reduction for the features before processing the data to the models has been done before the model 

Split the train data to train and validation

In [244]:

Y = train_data['Survived']
features_2 = train_data.drop(['Survived','Name','Cabin','Embarked','CabinLetter'], axis=1)

train_data.head()
X_train, X_val, Y_train , Y_val = train_test_split(features_2, Y, test_size=0.2, random_state= None)

#X_train.info()
Y_train.info()
#X_val.info()
Y_val.info()


<class 'pandas.core.series.Series'>
Int64Index: 712 entries, 448 to 869
Series name: Survived
Non-Null Count  Dtype
--------------  -----
712 non-null    int64
dtypes: int64(1)
memory usage: 11.1 KB
<class 'pandas.core.series.Series'>
Int64Index: 179 entries, 764 to 572
Series name: Survived
Non-Null Count  Dtype
--------------  -----
179 non-null    int64
dtypes: int64(1)
memory usage: 2.8 KB


Split test data 

In [245]:


X_test = test_data.drop(['Name','Cabin','Embarked'], axis=1)

test_data.head()

X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex          418 non-null    int64  
 3   Age          418 non-null    float64
 4   SibSp        418 non-null    int64  
 5   Parch        418 non-null    int64  
 6   Ticket       418 non-null    int64  
 7   Fare         418 non-null    float64
 8   AgeGroup     418 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 29.5 KB


Bayes Classifier 

In [246]:
Bayes_Classifier_Titanic = GaussianNB()
Bayes_Classifier_Titanic.fit(X_train,Y_train)


GaussianNB()

In [247]:

Bayes_pred = Bayes_Classifier_Titanic.predict(X_val)

acc= accuracy_score(Y_val, Bayes_pred)
bayes_precision = precision_score(Y_val, Bayes_pred)
bayes_recall = recall_score(Y_val, Bayes_pred)
bayes_f1 = f1_score(Y_val, Bayes_pred)
bayes_probs = Bayes_Classifier_Titanic.predict_proba(X_val)[:, 1]
bayes_roc_auc = roc_auc_score(Y_val, bayes_probs)
bayes_fpr, bayes_tpr, _ = roc_curve(Y_val, bayes_probs)


print("Bayes Metrics:")
print(f"Precision: {bayes_precision}")
print(f"Recall: {bayes_recall}")
print(f"F1-Score: {bayes_f1}")
print(f"ROC AUC: {bayes_roc_auc}")
print(f"Accuracy: {acc}")


conf_matrix = confusion_matrix(Y_val, Bayes_pred)
print("Confusion Matrix:\n", conf_matrix)



Bayes Metrics:
Precision: 0.7352941176470589
Recall: 0.6756756756756757
F1-Score: 0.7042253521126761
ROC AUC: 0.7960102960102959
Accuracy: 0.7653631284916201
Confusion Matrix:
 [[87 18]
 [24 50]]


Use model Bernoulli Naive Bayes

In [248]:

# Create a Bernoulli Naive Bayes classifier
BB_titanic = BernoulliNB(alpha=1.0)
BB_titanic.fit(X_train,Y_train)



BernoulliNB()

In [249]:
Bayes_pred_2 = BB_titanic.predict(X_val)

acc_2= accuracy_score(Y_val, Bayes_pred_2 )
Bbayes_precision = precision_score(Y_val, Bayes_pred_2)
Bbayes_recall = recall_score(Y_val, Bayes_pred_2)
Bbayes_f1 = f1_score(Y_val, Bayes_pred_2)
Bbayes_probs = BB_titanic.predict_proba(X_val)[:, 1]
Bbayes_roc_auc = roc_auc_score(Y_val, Bbayes_probs)
Bbayes_fpr, Bbayes_tpr, _ = roc_curve(Y_val, Bbayes_probs)


print("Bayes Metrics:")
print(f"Precision: {Bbayes_precision}")
print(f"Recall: {Bbayes_recall}")
print(f"F1-Score: {Bbayes_f1}")
print(f"ROC AUC: {Bbayes_roc_auc}")
print(f"Accuracy: {acc_2}")


conf_matrix = confusion_matrix(Y_val, Bayes_pred_2)
print("Confusion Matrix:\n", conf_matrix)

Bayes Metrics:
Precision: 0.8181818181818182
Recall: 0.6081081081081081
F1-Score: 0.6976744186046513
ROC AUC: 0.7858429858429858
Accuracy: 0.7821229050279329
Confusion Matrix:
 [[95 10]
 [29 45]]


Test the bayes classifier 

Upload the comparision data 

In [250]:
submission_data = pd.read_csv("titanic/gender_submission.csv")
Y_comp = submission_data["Survived"]

In [251]:
bayes_predictions =Bayes_Classifier_Titanic.predict(X_test)
acc_test_2 =accuracy_score( Y_comp,bayes_predictions)


print (bayes_predictions)

print(f"Accuracy: {acc_test_2}")

Accuracy: 0.8660287081339713
[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 1
 1 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 1
 1 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 1 0 1 0 1 0 0 1 0 0 0]


In [252]:
Bbayes_predictions =BB_titanic.predict(X_test)
acc_test_3 =accuracy_score( Y_comp,Bbayes_predictions)

print (bayes_predictions)

print(f"Accuracy: {acc_test_3}")

Accuracy: 1.0
[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0 1 0 0 1
 1 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 1
 1 1 0 1 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 1 0 1 0 1 0 0 1 0 0 0]
